In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('C:\machihe learning\Projects\Recomendation system\src\products.csv')

In [3]:
df.head(3)

,product_id,product_name,product_category,product_description,price
0,1,Follow Power,Toys & Games,Upon improve must put. Billion network left ac...,350.24
1,2,International Mr,Toys & Games,War three toward charge. Society event into me...,81.60
2,3,However Fish,Books,Arrive investment stage firm toward stay. Prod...,495.15


In [4]:
# Load the datasets
try:
    interactions_df = pd.read_csv('C:\\machihe learning\\Projects\\Recomendation system\\src\\interactions.csv')
    products_df = pd.read_csv('C:\\machihe learning\\Projects\\Recomendation system\\src\\products.csv')
    users_df = pd.read_csv('C:\\machihe learning\\Projects\\Recomendation system\\src\\users.csv')
    print("Data loaded successfully.")
except FileNotFoundError:
    print("Error: One or more CSV files not found. Make sure they are in the correct directory.")

Data loaded successfully.


In [5]:
# --- Perform initial inspection ---
print("\nInteractions DataFrame Info:")
interactions_df.info()


Interactions DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   user_id     100000 non-null  int64 
 1   product_id  100000 non-null  int64 
 2   rating      100000 non-null  int64 
 3   timestamp   100000 non-null  object
dtypes: int64(3), object(1)
memory usage: 3.1+ MB


In [6]:
# --- Merge the DataFrames ---
# We'll merge interactions with products to get product details
merged_df = pd.merge(interactions_df, products_df, on='product_id')

In [7]:
# Now, we'll merge with users to get user details
final_df = pd.merge(merged_df, users_df, on='user_id')

In [47]:
interactions_with_ratings_df = final_df.copy()

In [23]:
import os
os.makedirs('data/processed', exist_ok=True)
final_df.to_csv('data/processed/final_df.csv', index=False)
print('final_df saved to data/processed/final_df.csv')

final_df saved to data/processed/final_df.csv


In [24]:
print("\nFinal Merged DataFrame Info:")
final_df.info()


Final Merged DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 9 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   user_id              100000 non-null  int64  
 1   product_id           100000 non-null  int64  
 2   rating               100000 non-null  int64  
 3   timestamp            100000 non-null  object 
 4   product_name         100000 non-null  object 
 5   product_category     100000 non-null  object 
 6   product_description  100000 non-null  object 
 7   price                100000 non-null  float64
 8   signup_date          100000 non-null  object 
dtypes: float64(1), int64(3), object(5)
memory usage: 6.9+ MB


In [25]:
import os

os.makedirs('data/processed', exist_ok=True)
final_df.to_csv('data/processed/final_df.csv', index=False)

In [26]:
final_df.head(3)

,user_id,product_id,rating,timestamp,product_name,product_category,product_description,price,signup_date
0,279,1448,1,2024-01-01 07:47:50,Full Act,Apparel,Including experience just side. Meet factor in...,132.54,2023-12-08
1,385,1906,1,2024-01-12 19:14:44,Medical Catch,Sports,Test former major all. Once support political....,361.64,2024-05-23
2,740,2239,1,2024-05-11 08:04:50,Machine Exactly,Sports,Himself citizen mother. Age born south already...,340.67,2021-08-29


# Build the model

In [27]:
pip install scikit-surprise

You should consider upgrading via the 'c:\machihe learning\Projects\Recomendation system\recom39\Scripts\python.exe -m pip install --upgrade pip' command.


In [28]:
import pandas as pd
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy

In [44]:
!cd

c:\machihe learning\Projects\Recomendation system\notebooks


In [46]:
# Load the processed data
purchases_df = pd.read_csv('data/processed/final_df.csv')

In [48]:
reader = Reader(rating_scale=(1, 5)) 

In [49]:
# Load the DataFrame into the Surprise Dataset format
data = Dataset.load_from_df(purchases_df[['user_id', 'product_id', 'rating']], reader)

# Train the model

In [50]:
trainset = data.build_full_trainset()
model = SVD()
model.fit(trainset)

print("Model training complete.")

Model training complete.


In [33]:
# --- 3. Make a prediction for a specific user ---

In [51]:
# Let's pick a random user to make a prediction for.
user_id_to_predict = purchases_df['user_id'].iloc[0] # Get the first user from the DataFrame
product_id_to_predict = products_df[~products_df['product_id'].isin(
    purchases_df[purchases_df['user_id'] == user_id_to_predict]['product_id']
)]['product_id'].iloc[0]

# Predict the rating the user would give to the product
prediction = model.predict(user_id_to_predict, product_id_to_predict)

print(f"\nPredicted rating for user {user_id_to_predict} on product {product_id_to_predict}: {prediction.est}")


Predicted rating for user 279 on product 1: 1.0854945740261956


In [35]:
# --- 4. Get Top N recommendations for a user ---
# This is a more practical application.
def get_top_n_recommendations(model, user_id, all_products, k=10):
    # Get products the user has NOT purchased
    user_purchased_items = purchases_df[purchases_df['user_id'] == user_id]['product_id']
    unseen_products = [product for product in all_products if product not in user_purchased_items.values]

    # Predict ratings for all unseen products
    predictions = [model.predict(user_id, product_id) for product_id in unseen_products]

    # Sort the predictions by estimated rating
    predictions.sort(key=lambda x: x.est, reverse=True)

    # Get the top k products
    top_k_products = predictions[:k]
    
    return [pred.iid for pred in top_k_products]

# Get all unique product IDs
all_products = products_df['product_id'].unique()

top_recommendations = get_top_n_recommendations(model, user_id_to_predict, all_products)
print(f"\nTop 10 recommended product IDs for user {user_id_to_predict}:")
print(top_recommendations)


Top 10 recommended product IDs for user 952:
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


# Evaluating

In [52]:
# Split the data into training and testing sets
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

In [53]:
# --- Train the model on the training set ---
model = SVD()
model.fit(trainset)

In [54]:
# --- Make predictions on the testing set ---
predictions = model.test(testset)

In [55]:
rmse = accuracy.rmse(predictions)
print(f"The model's RMSE on the test set is: {rmse}")

RMSE: 1.3919
The model's RMSE on the test set is: 1.3919246753163164


In [56]:
from collections import defaultdict

def get_top_n(predictions, n=10):
    """Return the top-N recommendation for each user from a set of predictions."""
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]
    return top_n

top_n = get_top_n(predictions, n=10)

In [57]:
# Get all unique product IDs for recommendations
all_products = products_df['product_id'].unique()

In [58]:
# Choose a user to make recommendations for (e.g., the first user in the DataFrame)
user_id_to_predict = interactions_with_ratings_df['user_id'].iloc[0]

In [60]:
# Get the top 10 recommended product IDs
top_recommendations = get_top_n(model.test(trainset.build_anti_testset()), n=10)[user_id_to_predict]

In [61]:
print(f"\nTop 10 recommended product IDs and estimated ratings for user {user_id_to_predict}:")
for product_id, est_rating in top_recommendations:
    print(f"Product ID: {product_id}, Estimated Rating: {est_rating:.4f}")


Top 10 recommended product IDs and estimated ratings for user 279:
Product ID: 445, Estimated Rating: 3.3705
Product ID: 331, Estimated Rating: 3.2567
Product ID: 3913, Estimated Rating: 3.2220
Product ID: 2631, Estimated Rating: 3.1247
Product ID: 1361, Estimated Rating: 3.0925
Product ID: 1261, Estimated Rating: 3.0844
Product ID: 2075, Estimated Rating: 3.0838
Product ID: 772, Estimated Rating: 3.0714
Product ID: 1537, Estimated Rating: 3.0359
Product ID: 4363, Estimated Rating: 3.0089
